# Association rule mining

In this notebook, you'll implement the basic pairwise association rule mining algorithm.

To keep the implementation simple, you will apply your implementation to a simplified dataset, namely, letters ("items") in words ("receipts" or "baskets"). Having finished that code, you will then apply that code to some grocery store market basket data. If you write the code well, it will not be difficult to reuse building blocks from the letter case in the basket data case.

In [1]:
import sys
print(f"=== Python version ===\n{sys.version}")

=== Python version ===
3.8.7 (default, Jan 25 2021, 11:14:52) 
[GCC 5.5.0 20171010]


## Problem definition

Let's say you have a fragment of text in some language. You wish to know whether there are association rules among the letters that appear in a word. In this problem:

- Words are "receipts"
- Letters within a word are "items"

You want to know whether there are _association rules_ of the form, $a \implies b$, where $a$ and $b$ are letters. You will write code to do that by calculating for each rule its _confidence_, $\mathrm{conf}(a \implies b)$. "Confidence" will be another name for an estimate of the conditional probability of $b$ given $a$, or $\mathrm{Pr}[b \,|\, a]$.

## Sample text input

Let's carry out this analysis on a "dummy" text fragment, which graphic designers refer to as the [_lorem ipsum_](https://en.wikipedia.org/wiki/Lorem_ipsum):

In [2]:
latin_text = """
Sed ut perspiciatis, unde omnis iste natus error sit
voluptatem accusantium doloremque laudantium, totam
rem aperiam eaque ipsa, quae ab illo inventore
veritatis et quasi architecto beatae vitae dicta
sunt, explicabo. Nemo enim ipsam voluptatem, quia
voluptas sit, aspernatur aut odit aut fugit, sed
quia consequuntur magni dolores eos, qui ratione
voluptatem sequi nesciunt, neque porro quisquam est,
qui dolorem ipsum, quia dolor sit amet consectetur
adipisci[ng] velit, sed quia non numquam [do] eius
modi tempora inci[di]dunt, ut labore et dolore
magnam aliquam quaerat voluptatem. Ut enim ad minima
veniam, quis nostrum exercitationem ullam corporis
suscipit laboriosam, nisi ut aliquid ex ea commodi
consequatur? Quis autem vel eum iure reprehenderit,
qui in ea voluptate velit esse, quam nihil molestiae
consequatur, vel illum, qui dolorem eum fugiat, quo
voluptas nulla pariatur?

At vero eos et accusamus et iusto odio dignissimos
ducimus, qui blanditiis praesentium voluptatum
deleniti atque corrupti, quos dolores et quas
molestias excepturi sint, obcaecati cupiditate non
provident, similique sunt in culpa, qui officia
deserunt mollitia animi, id est laborum et dolorum
fuga. Et harum quidem rerum facilis est et expedita
distinctio. Nam libero tempore, cum soluta nobis est
eligendi optio, cumque nihil impedit, quo minus id,
quod maxime placeat, facere possimus, omnis voluptas
assumenda est, omnis dolor repellendus. Temporibus
autem quibusdam et aut officiis debitis aut rerum
necessitatibus saepe eveniet, ut et voluptates
repudiandae sint et molestiae non recusandae. Itaque
earum rerum hic tenetur a sapiente delectus, ut aut
reiciendis voluptatibus maiores alias consequatur
aut perferendis doloribus asperiores repellat.
"""

print("First 100 characters:\n  {} ...".format(latin_text[:100]))

First 100 characters:
  
Sed ut perspiciatis, unde omnis iste natus error sit
voluptatem accusantium doloremque laudantium,  ...


**Exercise 0** (ungraded). Look up and read the translation of _lorem ipsum_!

**Data cleaning.** Like most data in the real world, this dataset is noisy. It has both uppercase and lowercase letters, words have repeated letters, and there are all sorts of non-alphabetic characters. For our analysis, we should keep all the letters and spaces (so we can identify distinct words), but we should ignore case and ignore repetition within a word.

For example, the eighth word of this text is "error." As an _itemset_, it consists of the three unique letters, $\{e, o, r\}$. That is, treat the word as a set, meaning you only keep the unique letters.

This itemset has three possible _itempairs_: $\{e, o\}$, $\{e, r\}$, and $\{o, r\}$.

> Since sets are unordered, note that we would regard $\{e, o\} = \{o, e\}$, which is why we say there are only three itempairs, rather than six.

Start by writing some code to help "clean up" the input.

**Exercise 1** (2 points). Complete the following function, `normalize_string(s)`. The input `s` is a string (`str` object). The function should return a new string with (a) all characters converted to lowercase and (b) all non-alphabetic, non-whitespace characters removed.

> _Clarification_. Scanning the sample text, `latin_text`, you may see things that look like special cases. For instance, `inci[di]dunt` and `[do]`. For these, simply remove the non-alphabetic characters and only separate the words if there is explicit whitespace.
>
> For instance, `inci[di]dunt` would become `incididunt` (as a single word) and `[do]` would become `do` as a standalone word because the original string has whitespace on either side. A period or comma without whitespace would, similarly, just be treated as a non-alphabetic character inside a word _unless_ there is explicit whitespace. So `e pluribus.unum basium` would become `e pluribusunum basium` even though your common-sense understanding might separate `pluribus` and `unum`.
>
> _Hint_. Regard as a whitespace character anything "whitespace-like." That is, consider not just regular spaces, but also tabs, newlines, and perhaps others. To detect whitespaces easily, look for a "high-level" function that can help you do so rather than checking for literal space characters.

In [3]:
### Define demo inputs
demo_s_ex1 = latin_text[:100]
print(demo_s_ex1)


Sed ut perspiciatis, unde omnis iste natus error sit
voluptatem accusantium doloremque laudantium, 


<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
sed ut perspiciatis unde omnis iste natus error sit
voluptatem accusantium doloremque laudantium
```
<!-- Include any shout outs here -->

In [4]:
def normalize_string(s):
    assert type (s) is str
    ###
    ### YOUR CODE HERE
    new_str = [c for c in s.lower() if c.isalpha() or c.isspace()]
#    print(new_str)
    new_str2 = "".join(new_str)
#    print(new_str2)
    return(new_str2)
    ###
    
# Demo:
print(normalize_string(demo_s_ex1))


sed ut perspiciatis unde omnis iste natus error sit
voluptatem accusantium doloremque laudantium 


<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 1. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [5]:
### test_cell_ex1
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_1', 
    'func': normalize_string, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        's':{
            'dtype':'str', # data type of param.
            'check_modified':False,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'str',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': True, # Ignored if dtype is not df
            'check_row_order': True, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b's63L2lglDfBJpcKzxpwcyy61HyKnJNBOJXl9BMyWhyo=', path='resource/asnlib/publicdata/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise
print('Passed! Please submit.')

Passed! Please submit.


**Exercise 2** (1 point). Implement the following function, `get_normalized_words(s)`. It takes as input a string `s` (i.e., a `str` object). It should normalize `s` and then return a list of its words. (That is, the function should not assume that the input `s` is normalized yet.)

In [6]:
### Define demo inputs

demo_s_ex2 = latin_text[:33]
demo_s_ex2

'\nSed ut perspiciatis, unde omnis '

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
['sed', 'ut', 'perspiciatis', 'unde', 'omnis']
```
<!-- Include any shout outs here -->

In [7]:
def get_normalized_words (s):
    assert type(s) is str
    ###
    ### YOUR CODE HERE
    normalized_words = ""
    normalized_words = normalize_string(s)
#    print(normalized_words)
    normalized_list = []
    normalized_list = normalized_words.split()
#    print(normalized_list)
    return normalized_list
    ###

# Demo:
print(get_normalized_words(demo_s_ex2))

['sed', 'ut', 'perspiciatis', 'unde', 'omnis']


<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 2. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [8]:
### test_cell_ex2

from tester_fw.testers import Tester

conf = {
    'case_file':'tc_2', 
    'func': get_normalized_words, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        's':{
            'dtype':'str', # data type of param.
            'check_modified':False,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': True, # Ignored if dtype is not df
            'check_row_order': True, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b's63L2lglDfBJpcKzxpwcyy61HyKnJNBOJXl9BMyWhyo=', path='resource/asnlib/publicdata/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

Passed! Please submit.


**Exercise 3** (2 points). Implement a function, `make_itemsets_unstructured_text(text)`. The input, `text`, is a string of unstructured text (like the `latin_text` example above). Your function should get the normalized words from the text, convert the characters of each word into an itemset and then return the list of all itemsets. These output itemsets should appear in the same order as their corresponding words in the input. You may find it helpful to call `get_normalized_words` in your solution.

In [9]:
### Define demo inputs
demo_text_ex3 = 'sed \tut, perspiciatis\n und.e omnis'

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
[{'d', 'e', 's'},
 {'t', 'u'},
 {'a', 'c', 'e', 'i', 'p', 'r', 's', 't'},
 {'d', 'e', 'n', 'u'},
 {'i', 'm', 'n', 'o', 's'}]
```
<!-- Include any shout outs here -->
> Because sets are unordered, different versions of Python may produce sets with whose element-ordering differs from what you see above. However, the sets themselves should be in this order in the output list, since that is the order in which the corresponding words were given.

In [10]:
def make_itemsets_unstructured_text(text):
    ###
    ### YOUR CODE HERE
    normalized_words = []
    normalized_words = get_normalized_words(text)
#    print(normalized_words)

    return [set(normalized_set) for normalized_set in normalized_words]
    ###
    
make_itemsets_unstructured_text(demo_text_ex3)

[{'d', 'e', 's'},
 {'t', 'u'},
 {'a', 'c', 'e', 'i', 'p', 'r', 's', 't'},
 {'d', 'e', 'n', 'u'},
 {'i', 'm', 'n', 'o', 's'}]

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 3. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [11]:
### test_cell_ex3

from tester_fw.testers import Tester

conf = {
    'case_file':'tc_3', 
    'func': make_itemsets_unstructured_text, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'text':{
            'dtype':'list', # data type of param.
            'check_modified':True,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'list',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': True, # Ignored if dtype is not df
            'check_row_order': True, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b's63L2lglDfBJpcKzxpwcyy61HyKnJNBOJXl9BMyWhyo=', path='resource/asnlib/publicdata/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

Passed! Please submit.


## Implementing the basic algorithm

Recall the pseudocode for the algorithm that Rachel and Rich derived together:

![FindAssocRules (pseudocode)](https://ndownloader.figshare.com/files/7635700?private_link=3c473609741895a5cc2c)

In the following series of exercises, let's implement this method. We'll build it "bottom-up," first defining small pieces and working our way toward the complete algorithm. This method allows us to test each piece before combining them.

Observe that the bulk of the work in this procedure is just updating these tables, $T$ and $C$. So your biggest implementation decision is how to store those. A good choice is to use a dictionary

## Aside: Default dictionaries

Recall that the overall algorithm requires maintaining a table of item-pair (tuples) counts. It would be convenient to use a dictionary to store this table, where keys refer to item-pairs and the values are the counts.

However, with Python's built-in dictionaries, you always to have to check whether a key exists before updating it. For example, consider this code fragment:

```python
D = {'existing-key': 5} # Dictionary with one key-value pair

D['existing-key'] += 1 # == 6
D['new-key'] += 1  # Error: 'new-key' does not exist!
```

The second attempt causes an error because `'new-key'` is not yet a member of the dictionary. So, a more correct approach would be to do the following:

```python
D = {'existing-key': 5} # Dictionary with one key-value pair

if 'existing-key' not in D:
    D['existing-key'] = 0
D['existing-key'] += 1
   
if 'new-key' not in D:
    D['new-key'] = 0
D['new-key'] += 1
```

This pattern is so common that there is a special form of dictionary, called a _default dictionary_, which is available from the `collections` module: [`collections.defaultdict`](https://docs.python.org/3/library/collections.html?highlight=defaultdict#collections.defaultdict).

When you create a default dictionary, you need to provide a "factory" function that the dictionary can use to create an initial value when the key does *not* exist. For instance, in the preceding example, when the key was not present the code creates a new key with the initial value of an integer zero (0). Indeed, this default value is the one you get when you call `int()` with no arguments:

In [12]:
print(int())

0


In [13]:
from collections import defaultdict

D2 = defaultdict(int) # Empty dictionary

D2['existing-key'] = 5 # Create one key-value pair

D2['existing-key'] += 1 # Update
D2['new-key'] += 1

print(D2)

defaultdict(<class 'int'>, {'existing-key': 6, 'new-key': 1})


**Exercise 4** (2 points). Start by implementing a function that enumerates all item-pairs within an itemset and updates, _in-place_, a table that tracks the counts of those item-pairs.

The signature of this function is:

```python
   def update_pair_counts(pair_counts, itemset):
       ...
```

where `pair_counts` is the table to update and `itemset` is the itemset from which you need to enumerate item-pairs. You may assume `pair_counts` is a default dictionary. Each key is a pair of items `(a, b)`, and each value is the count. You may assume all items in `itemset` are distinct, i.e., that you may treat it as you would any set-like collection. Since the function will modify `pair_counts`, it does not need to return an object.

In [14]:
### Define demo inputs
demo_itemset_ex4 = {'f', 'r', 'o', 'g'}
demo_pair_counts_ex4 = defaultdict(int)
demo_pair_counts_ex4.update(
            {('o', 'e'): 1,
             ('e', 'o'): 1,
             ('o', 'r'): 1,
             ('r', 'o'): 1,
             ('e', 'r'): 1,
             ('r', 'e'): 1})

# This wrapper will return the updated pair_counts as a new object without modifying the original. 
# It's convenient for testing!
def update_pair_counts_wrapper(pair_counts, itemset):
    from copy import deepcopy
    pair_counts_cp = deepcopy(pair_counts)
    update_pair_counts(pair_counts_cp, itemset)
    return pair_counts_cp

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
defaultdict(int,
            {('o', 'e'): 1,
             ('e', 'o'): 1,
             ('o', 'r'): 2,
             ('r', 'o'): 2,
             ('e', 'r'): 1,
             ('r', 'e'): 1,
             ('f', 'o'): 1,
             ('o', 'f'): 1,
             ('f', 'r'): 1,
             ('r', 'f'): 1,
             ('f', 'g'): 1,
             ('g', 'f'): 1,
             ('o', 'g'): 1,
             ('g', 'o'): 1,
             ('r', 'g'): 1,
             ('g', 'r'): 1})
```
> Note: This displayed output is `demo_pair_counts_ex4` which was updated _in place_. Your solution does not need to return any object.

In [15]:
from collections import defaultdict
import itertools # Hint!
from itertools import combinations

def update_pair_counts (pair_counts, itemset):
    """
    Updates a dictionary of pair counts for
    all pairs of items in a given itemset.
    """
    assert type (pair_counts) is defaultdict

    ###
    ### YOUR CODE HERE
    # Keys will be a and b. Basically given a is b there.
#    print("pair counts = ", pair_counts)
#    print("itemset = ", itemset)
    
    #Displaying how the combinations function works
    
    sample_list = []
    sample_list = list(combinations(itemset,2))
#    print("sample list = ", sample_list)
    
    for (a,b) in combinations(itemset,2):
        pair_counts[(a,b)] += 1
        pair_counts[(b,a)] += 1
#    print("Return pair counts = ", pair_counts)    
    return pair_counts       
    ###
update_pair_counts_wrapper(demo_pair_counts_ex4, demo_itemset_ex4)


defaultdict(int,
            {('o', 'e'): 1,
             ('e', 'o'): 1,
             ('o', 'r'): 2,
             ('r', 'o'): 2,
             ('e', 'r'): 1,
             ('r', 'e'): 1,
             ('g', 'r'): 1,
             ('r', 'g'): 1,
             ('g', 'f'): 1,
             ('f', 'g'): 1,
             ('g', 'o'): 1,
             ('o', 'g'): 1,
             ('r', 'f'): 1,
             ('f', 'r'): 1,
             ('f', 'o'): 1,
             ('o', 'f'): 1})

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 4. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [16]:
### test_cell_ex4

from tester_fw.testers import Tester

conf = {
    'case_file':'tc_4', 
    'func': update_pair_counts_wrapper, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'pair_counts':{
            'dtype':'defaultdict', # data type of param.
            'check_modified':True,
        },
        'itemset':{
            'dtype':'set', # data type of param.
            'check_modified':False,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'defaultdict',
            'check_dtype': False,
            'check_col_dtypes': False, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': False, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b's63L2lglDfBJpcKzxpwcyy61HyKnJNBOJXl9BMyWhyo=', path='resource/asnlib/publicdata/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise


print('Passed! Please submit.')

Passed! Please submit.


**Exercise 5** (2 points). Implement a procedure that, given an itemset, updates a table to track counts of each item.

As with the previous exercise, you may assume all items in the given itemset (`itemset`) are distinct, i.e., that you may treat it as you would any set-like collection. You may also assume the table (`item_counts`) is a default dictionary.

In [17]:
### Define demo inputs
demo_item_counts_ex5 = defaultdict(int)
demo_item_counts_ex5.update({'o': 1, 'e': 1, 'r': 1})
demo_itemset_ex5 = {'f', 'r', 'o', 'g'}

def update_item_counts_wrapper(item_counts, itemset):
    from copy import deepcopy
    item_counts_cp = deepcopy(item_counts)
    update_item_counts(item_counts_cp, itemset)
    return item_counts_cp

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
defaultdict(int, {'o': 2, 'e': 1, 'r': 2, 'f': 1, 'g': 1})
```
<!-- Include any shout outs here -->

In [18]:
def update_item_counts(item_counts, itemset):
    ###
    ### YOUR CODE HERE
    #a is a Key
    #Increment item counts by each key of a, for example the key r is 2
    for (a) in itemset:
        item_counts[a] += 1
    
#    print('Item counts = ', item_counts)
    return item_counts
    ###
update_item_counts_wrapper(demo_item_counts_ex5, demo_itemset_ex5)

defaultdict(int, {'o': 2, 'e': 1, 'r': 2, 'g': 1, 'f': 1})

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 5. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [19]:
### test_cell_ex5

from tester_fw.testers import Tester

conf = {
    'case_file':'tc_5', 
    'func': update_item_counts_wrapper, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'item_counts':{
            'dtype':'defaultdict', # data type of param.
            'check_modified':False,
        },
        'itemset':{
            'dtype':'set', # data type of param.
            'check_modified':False,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'defaultdict',
            'check_dtype': False,
            'check_col_dtypes': False, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': False, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b's63L2lglDfBJpcKzxpwcyy61HyKnJNBOJXl9BMyWhyo=', path='resource/asnlib/publicdata/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

Passed! Please submit.


**Exercise 6** (2 points). Define `create_rules_from_counts` as follows: 
Given tables of item-pair counts (`pair_counts`) and individual item counts (`item_counts`) (You can assume both are default dictionaries), return all of the rules. The returned rules should be in the form of a dictionary whose key is the tuple, $(a, b)$ corresponding to the rule $a \Rightarrow b$, and whose value is the confidence of the rule, $\mathrm{conf}(a \Rightarrow b)$. 

You may assume that if $(a, b)$ is in the table of item-pair counts, then both $a$ and $b$ are in the table of individual item counts.

In [20]:
### Define demo inputs

demo_item_counts_ex7 = {'blue fish': 44, 'one fish': 47, 'red fish': 100, 'two fish': 74}
demo_pair_counts_ex7 = {('blue fish', 'one fish'): 16,
                        ('one fish', 'blue fish'): 16,
                        ('blue fish', 'red fish'): 36,
                        ('red fish', 'blue fish'): 36,
                        ('blue fish', 'two fish'): 27,
                        ('two fish', 'blue fish'): 27,
                        ('one fish', 'red fish'): 38,
                        ('red fish', 'one fish'): 38,
                        ('one fish', 'two fish'): 28,
                        ('two fish', 'one fish'): 28,
                        ('red fish', 'two fish'): 59,
                        ('two fish', 'red fish'): 59}

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
{('blue fish', 'one fish'): 0.36363636363636365,
 ('one fish', 'blue fish'): 0.3404255319148936,
 ('blue fish', 'red fish'): 0.8181818181818182,
 ('red fish', 'blue fish'): 0.36,
 ('blue fish', 'two fish'): 0.6136363636363636,
 ('two fish', 'blue fish'): 0.36486486486486486,
 ('one fish', 'red fish'): 0.8085106382978723,
 ('red fish', 'one fish'): 0.38,
 ('one fish', 'two fish'): 0.5957446808510638,
 ('two fish', 'one fish'): 0.3783783783783784,
 ('red fish', 'two fish'): 0.59,
 ('two fish', 'red fish'): 0.7972972972972973}
```
<!-- Include any shout outs here -->
> Note: The items in the "counts" dictionaries are phrases, not just letters! You don't need any special logic to handle this. It's just something to notice.  
  
> Note: If you did the division indirectly your result might not match "exactly". There's more in other notebooks on why that is. However, we have accounted for it in the testing, so don't worry as long as you're "reasonably close".

In [21]:
def create_rules_from_counts(pair_counts, item_counts):
    rules = {} # (item_a, item_b) -> conf (item_a => item_b)
    ###
    ### YOUR CODE HERE
    for (a,b) in pair_counts:
        assert a in item_counts
        confidence_ab = pair_counts[(a,b)]/ item_counts[a]
        rules[(a,b)] = confidence_ab
    ###
    return rules
rules = create_rules_from_counts(demo_pair_counts_ex7, demo_item_counts_ex7)
rules

{('blue fish', 'one fish'): 0.36363636363636365,
 ('one fish', 'blue fish'): 0.3404255319148936,
 ('blue fish', 'red fish'): 0.8181818181818182,
 ('red fish', 'blue fish'): 0.36,
 ('blue fish', 'two fish'): 0.6136363636363636,
 ('two fish', 'blue fish'): 0.36486486486486486,
 ('one fish', 'red fish'): 0.8085106382978723,
 ('red fish', 'one fish'): 0.38,
 ('one fish', 'two fish'): 0.5957446808510638,
 ('two fish', 'one fish'): 0.3783783783783784,
 ('red fish', 'two fish'): 0.59,
 ('two fish', 'red fish'): 0.7972972972972973}

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 6. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [22]:
### test_cell_ex6

from tester_fw.testers import Tester

conf = {
    'case_file':'tc_6', 
    'func': create_rules_from_counts, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'pair_counts':{
            'dtype':'dict', # data type of param.
            'check_modified':True,
        },
        'item_counts':{
            'dtype':'dict', # data type of param.
            'check_modified':True,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'dict',
            'check_dtype': True,
            'check_col_dtypes': False, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': False, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b's63L2lglDfBJpcKzxpwcyy61HyKnJNBOJXl9BMyWhyo=', path='resource/asnlib/publicdata/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

Passed! Please submit.


**Aside: pretty printing the rules.** The output of rules above is a little messy; here's a little helper function that structures that output a little, which will be useful for both debugging and reporting purposes.

In [23]:

def gen_rule_str(a, b, val=None, val_fmt='{:.3f}', sep=" = "):
    text = "{} => {}".format(a, b)
    if val:
        text = "conf(" + text + ")"
        text += sep + val_fmt.format(val)
    return text

def print_rules(rules):
    if type(rules) is dict or type(rules) is defaultdict:
        from operator import itemgetter
        ordered_rules = sorted(rules.items(), key=itemgetter(1), reverse=True)
    else: # Assume rules is iterable
        ordered_rules = [((a, b), None) for a, b in rules]
    for (a, b), conf_ab in ordered_rules:
        print(gen_rule_str(a, b, conf_ab))

# Demo:
print_rules(rules)

conf(blue fish => red fish) = 0.818
conf(one fish => red fish) = 0.809
conf(two fish => red fish) = 0.797
conf(blue fish => two fish) = 0.614
conf(one fish => two fish) = 0.596
conf(red fish => two fish) = 0.590
conf(red fish => one fish) = 0.380
conf(two fish => one fish) = 0.378
conf(two fish => blue fish) = 0.365
conf(blue fish => one fish) = 0.364
conf(red fish => blue fish) = 0.360
conf(one fish => blue fish) = 0.340


**Exercise 7** (1 Point). Given `rules`, a dictionary mapping pairs `(a, b)` to the confidence that `a` implies `b` as well as a `threshold`, define the function `filter_rules_by_conf`. It should return all the rules whose confidence is _at least_ the threshold.

In [24]:
### Define demo inputs

demo_rules_ex7 = {('blue fish', 'one fish'): 0.36363636363636365,
                    ('one fish', 'blue fish'): 0.3404255319148936,
                    ('blue fish', 'red fish'): 0.8181818181818182,
                    ('red fish', 'blue fish'): 0.36,
                    ('blue fish', 'two fish'): 0.6136363636363636,
                    ('two fish', 'blue fish'): 0.36486486486486486,
                    ('one fish', 'red fish'): 0.8085106382978723,
                    ('red fish', 'one fish'): 0.38,
                    ('one fish', 'two fish'): 0.5957446808510638,
                    ('two fish', 'one fish'): 0.3783783783783784,
                    ('red fish', 'two fish'): 0.59,
                    ('two fish', 'red fish'): 0.7972972972972973}
demo_threshold_ex7 = 0.59

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
{('blue fish', 'red fish'): 0.8181818181818182,
 ('blue fish', 'two fish'): 0.6136363636363636,
 ('one fish', 'red fish'): 0.8085106382978723,
 ('one fish', 'two fish'): 0.5957446808510638,
 ('red fish', 'two fish'): 0.59,
 ('two fish', 'red fish'): 0.7972972972972973}
```
<!-- Include any shout outs here -->

In [25]:
def filter_rules_by_conf(rules, threshold):
    ###
    ### YOUR CODE HERE
    # Create the new rules table
    rules2 = {}
    
#    print('Rules = ', rules)
#    print('Threshold = ', threshold)
    
#    for (key1, key2) in rules:
#        print('Key 1 = ', key1)
#        print('Key 2 = ', key2)
        
#    for value in rules.values():
#        print('Value = ', value)
#        if value >= threshold:
#            print('Greater than')
            
#    for i in rules:
#        print(i, rules[i])
#        if rules[i] >= threshold:
#            print('Greater than threshold')
            
    for k,v in rules.items():
#        print('Key = ', k)
#        print('Value = ',v)
        if v >= threshold:
#            print('Greater than threshold')
            rules2[k] = v
#    print('Updated Rules = ', rules2) 
#    print(type(rules2))
    return(rules2)
        
filter_rules_by_conf(demo_rules_ex7, demo_threshold_ex7)

{('blue fish', 'red fish'): 0.8181818181818182,
 ('blue fish', 'two fish'): 0.6136363636363636,
 ('one fish', 'red fish'): 0.8085106382978723,
 ('one fish', 'two fish'): 0.5957446808510638,
 ('red fish', 'two fish'): 0.59,
 ('two fish', 'red fish'): 0.7972972972972973}

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 7. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [26]:
### test_cell_ex7
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_7', 
    'func': filter_rules_by_conf, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'rules':{
            'dtype':'dict', # data type of param.
            'check_modified':True,
        },
        'threshold':{
            'dtype':'float', # data type of param.
            'check_modified':False,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'dict',
            'check_dtype': True,
            'check_col_dtypes': False, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': False, # Ignored if dtype is not df
            'float_tolerance': 0
        }
    }
}
tester = Tester(conf, key=b's63L2lglDfBJpcKzxpwcyy61HyKnJNBOJXl9BMyWhyo=', path='resource/asnlib/publicdata/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

Passed! Please submit.


## Actual baskets!

Let's take a look at some real data that [someone](http://www.salemmarafi.com/code/market-basket-analysis-with-r/) was kind enough to prepare for a similar exercise designed for the R programming environment.

First, here's a code snippet to load the data, which is a text file. If you are running in the Vocareum environment, we've already placed a copy of the data there; if you are running outside, this code will try to download a copy from the CSE 6040 website.

Each line of this file is some customer's shopping basket. The items that the customer bought are stored as a comma-separated list of values.

In [27]:
def on_vocareum():
    import os
    return os.path.exists('.voc')

def download(file, local_dir="", url_base=None, checksum=None):
    import os, requests, hashlib, io
    local_file = "{}{}".format(local_dir, file)
    if not os.path.exists(local_file):
        if url_base is None:
            url_base = "https://cse6040.gatech.edu/datasets/"
        url = "{}{}".format(url_base, file)
        print("Downloading: {} ...".format(url))
        r = requests.get(url)
        with open(local_file, 'wb') as f:
            f.write(r.content)            
    if checksum is not None:
        with io.open(local_file, 'rb') as f:
            body = f.read()
            body_checksum = hashlib.md5(body).hexdigest()
            assert body_checksum == checksum, \
                "Downloaded file '{}' has incorrect checksum: '{}' instead of '{}'".format(local_file,
                                                                                           body_checksum,
                                                                                           checksum)
    print("'{}' is ready!".format(file))
    
if on_vocareum():
    DATA_PATH = "./resource/asnlib/publicdata/"
else:
    DATA_PATH = ""
datasets = {'groceries.csv': '0a3d21c692be5c8ce55c93e59543dcbe'}

for filename, checksum in datasets.items():
    download(filename, local_dir=DATA_PATH, checksum=checksum)

with open('{}{}'.format(DATA_PATH, 'groceries.csv')) as fp:
    groceries_file = fp.read()
print (groceries_file[0:250] + "...\n... (etc.) ...") # Prints the first 250 characters only
print("\n(All data appears to be ready.)")

'groceries.csv' is ready!
citrus fruit,semi-finished bread,margarine,ready soups
tropical fruit,yogurt,coffee
whole milk
pip fruit,yogurt,cream cheese ,meat spreads
other vegetables,whole milk,condensed milk,long life bakery product
whole milk,butter,yogurt,rice,abrasive clea...
... (etc.) ...

(All data appears to be ready.)


**Exercise 8** (3 Points). The groceries data is unfortunately in a different format than the letters we have been working with. We can process itemsets well enough with `update_pair_counts`, `update_item_counts`, `create_rules_from_counts`, and `filter_rules_by_conf`. Making the itemsets themselves is a different story. If we're going to work with this real data set then we have to make itemsets out of it. 

**Your task**: Complete the function `make_itemsets_csv`. Given `csv_str`, a string where each receipt is a _line_ and the items within each receipt are _separated by a comma_, return a list of sets. Each set should be a single receipt, and each element of the set should be an item contained within that receipt. As with the words the ordering within each itemset does not matter, however the order of the itemsets within the list should match the order in which they appear.

In [28]:
### Define demo inputs

demo_csv_str_ex8 = '''milk,eggs,peanut butter,oatmeal
butter,pancake mix,maple syrup
dog treats,milk,milk'''
print(demo_csv_str_ex8)

milk,eggs,peanut butter,oatmeal
butter,pancake mix,maple syrup
dog treats,milk,milk


<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
[{'eggs', 'milk', 'oatmeal', 'peanut butter'},
 {'butter', 'maple syrup', 'pancake mix'},
 {'dog treats', 'milk'}]
```
<!-- Include any shout outs here -->

In [29]:
# ex8 solution
def make_itemsets_csv(csv_str):
    ###
    ### YOUR CODE HERE
#    print('CSV str = ', csv_str)
    #Using the splitlines function (not just split) split the string by new line
    x = csv_str.splitlines()
  #  print('X = ', x)
  #  print('X type = ', type(x))
    
    new_list = []
    count = 0
    
    for i in x:
        temp_list = []
#        print('I = ', i)
#        print('Type = ', type(i))
#       Split the list by commas
        temp_list = i.split(',')
#       Sort the list    
        temp_list.sort()
#        print('Temp list = ', temp_list)
#       Create a set to have unique values in each list
        temp_set = set(temp_list)
#        print('Type of Temp set = ', type(temp_set))
#       Add the set to the list that will be returned
        new_list.append(temp_set)


    
#    print('New List = ',new_list)
    
    return new_list
    ###
make_itemsets_csv(demo_csv_str_ex8)

[{'eggs', 'milk', 'oatmeal', 'peanut butter'},
 {'butter', 'maple syrup', 'pancake mix'},
 {'dog treats', 'milk'}]

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 8. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [30]:
### test_cell_ex8
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_8', 
    'func': make_itemsets_csv, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'csv_str':{
            'dtype':'str', # data type of param.
            'check_modified':False,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'',
            'check_dtype': True,
            'check_col_dtypes': False, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': False, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b's63L2lglDfBJpcKzxpwcyy61HyKnJNBOJXl9BMyWhyo=', path='resource/asnlib/publicdata/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

Passed! Please submit.


**On higher order functions**  
Python functions are *objects*, just like strings, lists, dictionaries, etc. This means that they can be passed as arguments to other functions. (You have seen this before with sorting in Notebook 1.) Here's a short example, say I have this function.

```
def do_math_operation(x, y, func):
    return func(x, y)
```

Then I can define other functions and pass them to `do_math_operation` to change how it works. For instance let's say I have `add(a, b)` which returns the sum of `a` and `b`. I can call `do_math_operation(4, 5, add)` and the result will be the sum of `4+5`. I could also define a funciton called a function which multiplies two numbers pass that to change what `do_math_operation` does. For sure, that's not a very interesting example, but this is a very powerful concept. 

**Exercise 9** (4 Points). Use the tools you have created above to create a association rules from a data source. The following inputs:  
- `source`: the source data for your rules. You can assume this will be _either_ structured like the `latin_text` _or_ like the csv-formatted `groceries_file`. 
- `itemset_maker`: a function which can be used to transform the `source` into "itemsets". *Since you are given the parsing function as a parameter, you do not need to attempt to determine the format of `source`*. See "On higher order functions" above.
- `conf_threshold` and `min_count`: Your result should only include rules where `a` occurs in _at least_ `min_count` receipts _and_ $\mathrm{conf}(a \Rightarrow b)$ is at least `conf_threshold`

Return your confidence rules as a dictionary.
- Keys: pairs `(a,b)`
- Values: $\mathrm{conf}(a \Rightarrow b)$

> Note: We will test your solution using `make_itemsets_unstructured_text` and `make_itemsets_csv` on random inputs. These functions must be correctly defined above to pass the test cell. 

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
Source: `latin_text`; Itemset Maker: `make_itemsets_unstructured_text`; Confidence Threshold: 0.75; Min Count: 0
conf(q => u) = 1.000
conf(x => e) = 1.000
conf(h => i) = 0.833
conf(x => i) = 0.833
conf(v => t) = 0.818
conf(r => e) = 0.800
conf(v => e) = 0.773
conf(b => i) = 0.750
conf(g => i) = 0.750
conf(f => i) = 0.750

Source: `groceries_file`; Itemset Maker: `make_itemsets_csv`; Confidence Threshold: 0.5; Min Count: 10
conf(honey => whole milk) = 0.733
conf(frozen fruits => other vegetables) = 0.667
conf(cereals => whole milk) = 0.643
conf(rice => whole milk) = 0.613
conf(rubbing alcohol => whole milk) = 0.600
conf(cocoa drinks => whole milk) = 0.591
conf(pudding powder => whole milk) = 0.565
conf(jam => whole milk) = 0.547
conf(cream => other vegetables) = 0.538
conf(cream => sausage) = 0.538
conf(baking powder => whole milk) = 0.523
conf(tidbits => rolls/buns) = 0.522
conf(rice => other vegetables) = 0.520
conf(cooking chocolate => whole milk) = 0.520
conf(specialty cheese => other vegetables) = 0.500
conf(rubbing alcohol => butter) = 0.500
conf(rubbing alcohol => citrus fruit) = 0.500
conf(ready soups => rolls/buns) = 0.500
conf(frozen fruits => whipped/sour cream) = 0.500
```
<!-- Include any shout outs here -->

> Note: The "Source: ...; Itemset Maker: ...; Confidence Threshold: ...; Min Count: ..." is just printed for convenience. It _should not be part of your output_!  
  
> Note: The rules are "pretty printed" using the `print_rules` function defined above. Your solution should output lists of dictionaries.  
  
> Note: The demo includes _two calls_ to your solution. The first is for `latin_rules`, and the second is for `groceries_file`.

In [31]:
def filter_rules_by_conf2(rules, threshold, mincount):
    ###
    ### YOUR CODE HERE
    # Create the new rules table
    rules2 = {}
    
#    print('Rules = ', rules)
#    print('Rules type = ', type(rules))
#    print('Threshold = ', threshold)
#    print('Min Count = ', mincount)
    
#    for (key1, key2) in rules:
#        print('Key 1 = ', key1)
#        print('Key 2 = ', key2)
        
#    for value in rules.values():
#        print('Value = ', value)
#        if value >= threshold:
#            print('Greater than')
            
#    for i in rules:
#        print(i, rules[i])
#        if rules[i] >= threshold:
#            print('Greater than threshold')
    count = 0        
    for k,v in rules.items():
#        print('Key = ', k)
#        print('Value = ',v)
        if v >= threshold:
#            print('Greater than threshold')
#            if rules.count(k[count]) >= mincount:
#                print('Min Count met')
            rules2[k] = v
#    print('Updated Rules = ', rules2) 
#    print(type(rules2))
    return(rules2)

def create_rules_from_source(source, itemset_maker, conf_threshold=0, min_count=0):
    ###
    ### YOUR CODE HERE

# Call functions to create pair_dict and item_counts; Remember that these have to be
# initialized as defaultdict first

#    print('Min count = ', min_count)
    pair_dict = defaultdict(int)
    item_count = defaultdict(int)
    
    itemsets = itemset_maker(source)
# Iterate through the itemsets to create the pair_counts and item_count default dictionaries

    for i in itemsets:
#        print('Itemsets = ', i)
        update_pair_counts(pair_dict,i)
        update_item_counts(item_count,i)
        
#    print('Ex 9 pair dict = ', pair_dict)
#    print('Ex 9 item count = ', item_count)


# Next get the confidence rules 

    source_rules = create_rules_from_counts(pair_dict, item_count)
#    print('Source Rules = ', source_rules)
    
    
    filtered_rules = filter_rules_by_conf2(source_rules, conf_threshold, min_count)
#    print('Filtered Rules = ', filtered_rules)

    return filtered_rules
    ###
    
latin_rules = create_rules_from_source(latin_text, make_itemsets_unstructured_text, 0.75)
grocery_rules = create_rules_from_source(groceries_file, make_itemsets_csv, 0.5, 10)
print('Source: `latin_text`; Itemset Maker: `make_itemsets_unstructured_text`; Confidence Threshold: 0.75; Min Count: 0')
print_rules(latin_rules)
print()
print('Source: `groceries_file`; Itemset Maker: `make_itemsets_csv`; Confidence Threshold: 0.5; Min Count: 10')
print_rules(grocery_rules)

Source: `latin_text`; Itemset Maker: `make_itemsets_unstructured_text`; Confidence Threshold: 0.75; Min Count: 0
conf(q => u) = 1.000
conf(x => e) = 1.000
conf(h => i) = 0.833
conf(x => i) = 0.833
conf(v => t) = 0.818
conf(r => e) = 0.800
conf(v => e) = 0.773
conf(b => i) = 0.750
conf(g => i) = 0.750
conf(f => i) = 0.750

Source: `groceries_file`; Itemset Maker: `make_itemsets_csv`; Confidence Threshold: 0.5; Min Count: 10
conf(baby food => pip fruit) = 1.000
conf(baby food => pasta) = 1.000
conf(baby food => white bread) = 1.000
conf(baby food => fruit/vegetable juice) = 1.000
conf(baby food => cake bar) = 1.000
conf(baby food => waffles) = 1.000
conf(baby food => sausage) = 1.000
conf(baby food => tropical fruit) = 1.000
conf(baby food => finished products) = 1.000
conf(baby food => yogurt) = 1.000
conf(baby food => curd) = 1.000
conf(baby food => pastry) = 1.000
conf(baby food => butter milk) = 1.000
conf(baby food => soft cheese) = 1.000
conf(baby food => frozen vegetables) = 1.000

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 9. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [32]:
### test_cell_ex9
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_9', 
    'func': create_rules_from_source, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'source':{
            'dtype':'string', # data type of param.
            'check_modified':False,
        },
        'itemset_maker':{
            'dtype':'function', # data type of param.
            'check_modified':False,
        },
        'conf_threshold':{
            'dtype':'float', # data type of param.
            'check_modified':False,
        },
        'min_count':{
            'dtype':'int', # data type of param.
            'check_modified':False,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'dict',
            'check_dtype': True,
            'check_col_dtypes': False, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': False, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b's63L2lglDfBJpcKzxpwcyy61HyKnJNBOJXl9BMyWhyo=', path='resource/asnlib/publicdata/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

AssertionError: 
Output for output_0 is incorrect.
The returned result is available as `returned_output_vars['output_0']`
The expected result is available as `true_output_vars['output_0']`
            

**Aside** In case you \*cough\* didn't get around to \*cough\* looking up and reading the translation of the Lorem Ipsum, here's the first two paragraphs. We're going to use it in the next exercise

In [33]:
english_text = """
But I must explain to you how all this mistaken idea
of denouncing of a pleasure and praising pain was
born and I will give you a complete account of the
system, and expound the actual teachings of the great
explorer of the truth, the master-builder of human
happiness. No one rejects, dislikes, or avoids
pleasure itself, because it is pleasure, but because
those who do not know how to pursue pleasure
rationally encounter consequences that are extremely
painful. Nor again is there anyone who loves or
pursues or desires to obtain pain of itself, because
it is pain, but occasionally circumstances occur in
which toil and pain can procure him some great
pleasure. To take a trivial example, which of us
ever undertakes laborious physical exercise, except
to obtain some advantage from it? But who has any
right to find fault with a man who chooses to enjoy
a pleasure that has no annoying consequences, or
one who avoids a pain that produces no resultant
pleasure?

On the other hand, we denounce with righteous
indignation and dislike men who are so beguiled and
demoralized by the charms of pleasure of the moment,
so blinded by desire, that they cannot foresee the
pain and trouble that are bound to ensue; and equal
blame belongs to those who fail in their duty
through weakness of will, which is the same as
saying through shrinking from toil and pain. These
cases are perfectly simple and easy to distinguish.
In a free hour, when our power of choice is
untrammeled and when nothing prevents our being
able to do what we like best, every pleasure is to
be welcomed and every pain avoided. But in certain
circumstances and owing to the claims of duty or
the obligations of business it will frequently
occur that pleasures have to be repudiated and
annoyances accepted. The wise man therefore always
holds in these matters to this principle of
selection: he rejects pleasures to secure other
greater pleasures, or else he endures pains to
avoid worse pains.
"""

**Exercise 10** (2 points). Let's consider the case when we have more than one "grocery list". We want to find the rules which are common to both. 

Given the following inputs complete the funtion `common_rules`:
- `source_0` and `source_1` - strings containing source data. These can be unstructured text, csv, or really anything that can be converted into "itemsets". You can assume that both are able to be processed by the same `itemset_maker`.
- `itemset_maker` - a function which will process a _single_ data source into itemsets.
- `conf_threshold` and `min_count`: Your result should only include rules where `a` occurs in _at least_ `min_count` receipts _and_ $\mathrm{conf}(a \Rightarrow b)$ is at least `conf_threshold`.

Your function should return a set of tuples, where each tuple is a key in both the rules generated from `source_0` and `source_1`.

> Note: We will test your solution using `make_itemsets_unstructured_text` and `make_itemsets_csv` on random inputs. These functions must be correctly defined above to pass the test cell. 

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
{('q', 'u'), ('x', 'e')}
```
These are the keys for the rules common to both the Latin and English texts.

In [34]:
# ex10 solution
def common_rules(source_0, source_1, itemset_maker, conf_threshold, min_count):
    ###
    ### YOUR CODE HERE
    pair_dict = defaultdict(int)
    pair_dict2 = defaultdict(int)
    item_count = defaultdict(int)
    item_count2 = defaultdict(int)
    
    itemsets = itemset_maker(source_0)
    itemsets2 = itemset_maker(source_1)
# Iterate through the itemsets to create the pair_counts and item_count default dictionaries

    for i in itemsets:
#        print('Itemsets = ', i)
        update_pair_counts(pair_dict,i)
        update_item_counts(item_count,i)
        
    for i in itemsets2:
        update_pair_counts(pair_dict2,i)
        update_item_counts(item_count2,i)
        
#    print('Ex 9 pair dict = ', pair_dict)
#    print('Ex 9 item count = ', item_count)


# Next get the confidence rules 

    source_rules = create_rules_from_counts(pair_dict, item_count)
    source_rules2 = create_rules_from_counts(pair_dict2, item_count2)
#    print('Source Rules = ', source_rules)
    
    
    filtered_rules = filter_rules_by_conf2(source_rules, conf_threshold, min_count)
    filtered_rules2 = filter_rules_by_conf2(source_rules2, conf_threshold, min_count)
    print('Filtered Rules = ', filtered_rules)
    print('Filtered Rules2 = ', filtered_rules2)

    for i in filtered_rules2.keys():
        filtered_rules[i] = filtered_rules2[i]
    print('Filtered Rules new = ', filtered_rules)
    return filtered_rules
    ###
common_rules(latin_text, english_text, make_itemsets_unstructured_text, 0.75, 0)

Filtered Rules =  {('r', 'e'): 0.8, ('v', 'e'): 0.7727272727272727, ('v', 't'): 0.8181818181818182, ('q', 'u'): 1.0, ('h', 'i'): 0.8333333333333334, ('x', 'e'): 1.0, ('b', 'i'): 0.75, ('x', 'i'): 0.8333333333333334, ('g', 'i'): 0.75, ('f', 'i'): 0.75}
Filtered Rules2 =  {('x', 'e'): 1.0, ('k', 'e'): 0.7777777777777778, ('j', 'e'): 1.0, ('q', 'e'): 1.0, ('q', 'n'): 0.75, ('q', 'u'): 1.0, ('z', 'e'): 1.0, ('z', 'l'): 1.0, ('z', 'o'): 1.0, ('z', 'i'): 1.0, ('z', 'a'): 1.0, ('z', 'd'): 1.0, ('z', 'r'): 1.0, ('z', 'm'): 1.0}
Filtered Rules new =  {('r', 'e'): 0.8, ('v', 'e'): 0.7727272727272727, ('v', 't'): 0.8181818181818182, ('q', 'u'): 1.0, ('h', 'i'): 0.8333333333333334, ('x', 'e'): 1.0, ('b', 'i'): 0.75, ('x', 'i'): 0.8333333333333334, ('g', 'i'): 0.75, ('f', 'i'): 0.75, ('k', 'e'): 0.7777777777777778, ('j', 'e'): 1.0, ('q', 'e'): 1.0, ('q', 'n'): 0.75, ('z', 'e'): 1.0, ('z', 'l'): 1.0, ('z', 'o'): 1.0, ('z', 'i'): 1.0, ('z', 'a'): 1.0, ('z', 'd'): 1.0, ('z', 'r'): 1.0, ('z', 'm'): 1.0

{('r', 'e'): 0.8,
 ('v', 'e'): 0.7727272727272727,
 ('v', 't'): 0.8181818181818182,
 ('q', 'u'): 1.0,
 ('h', 'i'): 0.8333333333333334,
 ('x', 'e'): 1.0,
 ('b', 'i'): 0.75,
 ('x', 'i'): 0.8333333333333334,
 ('g', 'i'): 0.75,
 ('f', 'i'): 0.75,
 ('k', 'e'): 0.7777777777777778,
 ('j', 'e'): 1.0,
 ('q', 'e'): 1.0,
 ('q', 'n'): 0.75,
 ('z', 'e'): 1.0,
 ('z', 'l'): 1.0,
 ('z', 'o'): 1.0,
 ('z', 'i'): 1.0,
 ('z', 'a'): 1.0,
 ('z', 'd'): 1.0,
 ('z', 'r'): 1.0,
 ('z', 'm'): 1.0}

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 10. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [35]:
### test_cell_ex10
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_10', 
    'func': common_rules, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'source_0':{
            'dtype':'str', # data type of param.
            'check_modified':False,
        },
        'source_1':{
            'dtype':'str', # data type of param.
            'check_modified':False,
        },
        'itemset_maker':{
            'dtype':'function', # data type of param.
            'check_modified':False,
        },
        'conf_threshold':{
            'dtype':'float', # data type of param.
            'check_modified':False,
        },
        'min_count':{
            'dtype':'str', # data type of param.
            'check_modified':False,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'set',
            'check_dtype': True,
            'check_col_dtypes': False, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': False, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b's63L2lglDfBJpcKzxpwcyy61HyKnJNBOJXl9BMyWhyo=', path='resource/asnlib/publicdata/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

Filtered Rules =  {('meat spreads', 'yogurt'): 0.625, ('rice', 'whole milk'): 0.5957446808510638, ('abrasive cleaner', 'other vegetables'): 0.5416666666666666, ('cereals', 'whole milk'): 0.6216216216216216, ('curd', 'whole milk'): 0.5033557046979866, ('baking powder', 'whole milk'): 0.49411764705882355, ('herbs', 'whole milk'): 0.51, ('herbs', 'other vegetables'): 0.51, ('skin care', 'whole milk'): 0.5882352941176471, ('curd cheese', 'whole milk'): 0.5, ('rice', 'other vegetables'): 0.5319148936170213, ('specialty cheese', 'other vegetables'): 0.5, ('rubbing alcohol', 'bottled water'): 0.5, ('rubbing alcohol', 'root vegetables'): 0.5, ('rubbing alcohol', 'whole milk'): 0.625, ('rubbing alcohol', 'citrus fruit'): 0.625, ('rubbing alcohol', 'butter'): 0.5, ('rubbing alcohol', 'napkins'): 0.5, ('rubbing alcohol', 'hygiene articles'): 0.5, ('ready soups', 'rolls/buns'): 0.5, ('ready soups', 'root vegetables'): 0.5, ('ready soups', 'whole milk'): 0.5, ('toilet cleaner', 'whole milk'): 0.5, 

AssertionError: Type <class 'set'> is required for output_0 but <class 'dict'> was returned.

**Fin.** If you have made it this far, congratulations on completing the assignment. **Don't forget to submit!**